In [ ]:
import panel as pn
import pandas as pd

pn.extension('vizzu', 'tabulator', template='bootstrap')
import hvplot.pandas

import holoviews as hv

In [ ]:
windturbines = pd.read_parquet('https://datasets.holoviz.org/windturbines/v1/windturbines.parq')

windturbines.head()

In [ ]:
ls = hv.link_selections.instance()


geo = ls(windturbines.hvplot.points(
    'easting', 'northing', xaxis=None, yaxis=None, rasterize=True, xlim=(-15000000, -5000000),
    tiles='ESRI', height=500, dynspread=True
))

In [ ]:
def data(df, groupby, quant):
    if quant == 'Count':
        return df.value_counts(groupby).to_frame(name='Count').sort_index().reset_index().iloc[:50]
    else:
        return df.groupby(groupby)[quant].sum().reset_index().iloc[:50]

def config(chart_type, groupby, quant):
    if chart_type == 'Bubble Chart':
        return {
            "channels": {
                "x": None,
                "y": None,
                "color": groupby,
                "label": groupby,
                "size": quant
            },
            'geometry': 'circle'
        }
    else:
        return {
            "channels": {
                "x": groupby,
                "y": quant,
                "color": None,
                "label": None,
                "size": None
            },
            'geometry': 'rectangle'
        }
    
groupby = pn.widgets.RadioButtonGroup(options={'State': 't_state', 'Year': 'p_year', 'Manufacturer': 't_manu'}, align='center')
chart_type = pn.widgets.RadioButtonGroup(options=['Bar Chart', 'Bubble Chart'], align='center')
quant = pn.widgets.RadioButtonGroup(options={'Count': 'Count', 'Capacity': 'p_cap'}, align='center')
lsdata = ls.selection_param(windturbines)

vizzu = pn.pane.Vizzu(
    pn.bind(data, lsdata, groupby, quant),
    config=pn.bind(config, chart_type, groupby, quant),
    column_types={'p_year': 'dimension'},
    style={
        "plot": {
            "xAxis": {
                "label": {
                    "angle": "-45deg"
                }
            }
        }
    },
    sizing_mode='stretch_both'
)

table = pn.widgets.Tabulator(lsdata, page_size=15, pagination='remote', sizing_mode='stretch_width')

In [ ]:
pn.Column(
    pn.Row(quant, "# by", groupby, "# as a", chart_type).servable(area='header'),
    pn.Column(
        pn.Row(geo, table),
        vizzu, min_height=1000,
        sizing_mode='stretch_both'
    ).servable(title='Windturbines')
)